# Formulate Model

## Generate Transitions

In [ ]:
from __future__ import annotations
from IPython.display import Math
from qrules.particle import load_pdg
from ampform.io import aslatex, improve_latex_rendering
import matplotlib.pyplot as plt

improve_latex_rendering()
particle_db = load_pdg()

In [ ]:
from qrules.particle import Particle, Spin

pgamma = Particle(
    name="pgamma",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma

In [ ]:
pgamma2 = Particle(
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99991,
)
pgamma2

In [ ]:
particle_db.add(pgamma)
particle_db.add(pgamma2)

In [ ]:
particle_db["N(1680)+"]

In [ ]:
particle_db["N(1650)+"]

In [ ]:
particle_db["Delta(1232)+"]

For simplicity, we limit the intermediate resonances to be:

"$a(2)$" for $\eta \pi^0$, 

Both "$N*$" for $\pi^0 p$
and $\eta p$

In [ ]:
import qrules
import graphviz

reaction1_strong_EM = qrules.generate_transitions(
    initial_state=("pgamma"),
    final_state=["eta", "pi0", "p"],
    allowed_intermediate_particles=["a(2)(1320)", "N(1440)", "Delta(1232)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    # mass_conservation_factor=0,
)

In [ ]:
dot_se = qrules.io.asdot(reaction1_strong_EM, collapse_graphs=True)
graphviz.Source(dot_se)

## Build Amplitude Models

we use ampform to formulate the transitions as an amplitude model (here: HelicityModel). 

In [ ]:
import ampform

model_builder = ampform.get_builder(reaction1_strong_EM)
model_no_dynamics = model_builder.formulate()
model_no_dynamics.intensity

In [ ]:
from ampform.io import aslatex

Math(aslatex(model_no_dynamics.amplitudes))

In [ ]:
Math(aslatex(model_no_dynamics.parameter_defaults))

We choose to use relativistic_breit_wigner_with_ff() as the lineshape for all resonances and use a Blatt-Weisskopf form factor (create_non_dynamic_with_ff()) for the production decay. 

In [ ]:
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.dynamics.assign("pgamma", create_non_dynamic_with_ff)
for name in reaction1_strong_EM.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

take another look at the parameters of the model to see which new parameters are there:

In [ ]:
sorted_parameter_defaults = {
    symbol: model.parameter_defaults[symbol]
    for symbol in sorted(model.parameter_defaults, key=str)
}
src = aslatex(sorted_parameter_defaults)
Math(src)

Optionally, we can backup the HelicityModel to disk via pickle.

In [ ]:
import pickle

qrules.io.write(reaction1_strong_EM, "transitions.json")
with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

# Generate data

In [ ]:
import pickle

from ampform.helicity import HelicityModel

with open("helicity_model.pickle", "rb") as model_file:
    imported_model: HelicityModel = pickle.load(model_file)

In [ ]:
initial_state, *_ = imported_model.reaction_info.initial_state.values()
print("Initial state:")
print(" ", initial_state.name)
print("Final state:")
for i, p in imported_model.reaction_info.final_state.items():
    print(f"  {i}: {p.name}")
del initial_state

## Generate phase space sample

In [ ]:
from tensorwaves.data import TFPhaseSpaceGenerator, TFUniformRealNumberGenerator

rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction1_strong_EM.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction1_strong_EM.final_state.items()},
)
phsp_momenta = phsp_generator.generate(1_000_000, rng)

In [ ]:
import numpy as np
import pandas as pd

pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in phsp_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

In [ ]:
import numpy as np

p = np.array(list(phsp_momenta.values()))
p.shape

In [ ]:
p.sum(axis=0).round(decimals=14)

In [ ]:
E0 = p[0].T[0]
px0 = p[0].T[1]
py0 = p[0].T[2]
pz0 = p[0].T[3]
p0 = np.sqrt(px0**2 + py0**2 + pz0**2)

E1 = p[1].T[0]
px1 = p[1].T[1]
py1 = p[1].T[2]
pz1 = p[1].T[3]
p1 = np.sqrt(px1**2 + py1**2 + pz1**2)

E2 = p[2].T[0]
px2 = p[2].T[1]
py2 = p[2].T[2]
pz2 = p[2].T[3]
p2 = np.sqrt(px2**2 + py2**2 + pz2**2)

In [ ]:
np.sqrt(px0**2 + py0**2 + pz0**2)

In [ ]:
phsp_momenta

## Generate intensity-based sample

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="numpy",
)

In [ ]:
Math(aslatex(model.kinematic_variables))

In [ ]:
from tensorwaves.data import SympyDataTransformer

helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)

In [ ]:
from tensorwaves.data import (
    IntensityDistributionGenerator,
    TFWeightedPhaseSpaceGenerator,
)

weighted_phsp_generator = TFWeightedPhaseSpaceGenerator(
    initial_state_mass=reaction1_strong_EM.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction1_strong_EM.final_state.items()},
)
data_generator = IntensityDistributionGenerator(
    domain_generator=weighted_phsp_generator,
    function=intensity_func,
    domain_transformer=helicity_transformer,
)
data_momenta = data_generator.generate(100_000, rng)
pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in data_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

## Visualize kinematic variables

In [ ]:
phsp = helicity_transformer(phsp_momenta)
data = helicity_transformer(data_momenta)
list(data)

In [ ]:
for state_id, particle in reaction1_strong_EM.final_state.items():
    print(f"ID {state_id}:", particle.name)

In [ ]:
import pandas as pd

data_frame = pd.DataFrame(data)
phsp_frame = pd.DataFrame(phsp)
data_frame

In [ ]:
plt.hist2d(phsp["m_01"] ** 2, phsp["m_12"] ** 2, bins=100, cmin=1)
plt.show()

In [ ]:
plt.hist2d(data["m_01"] ** 2, data["m_12"] ** 2, bins=100, cmin=1)
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction1_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(9, 4))
ax.hist(
    np.real(data_frame["m_12"]),
    bins=100,
    alpha=0.5,
    density=True,
)
ax.set_xlabel("$m$ [GeV]")
for p, color in zip(resonances, colors):
    ax.axvline(x=p.mass, linestyle="dotted", label=p.name, color=color)
ax.legend()
plt.show()

## Export data sets

In [ ]:
import pickle

with open("data.pickle", "wb") as stream:
    pickle.dump(data, stream)
with open("phsp.pickle", "wb") as stream:
    pickle.dump(phsp, stream)

# Perform fit

In [ ]:
reaction = qrules.io.load("transitions.json")
with open("helicity_model.pickle", "rb") as stream:
    model: HelicityModel = pickle.load(stream)
with open("data.pickle", "rb") as stream:
    data = pickle.load(stream)
with open("phsp.pickle", "rb") as stream:
    phsp = pickle.load(stream)

## Prepare parametrized function

In [ ]:
# initial_parameters = {
#     R"C_{p\gamma \to {f_{0}(1500)}_{0} \gamma_{+1}; f_{0}(1500) \to  \eta \pi^{0} p}": (
#         1.0 + 0.0j
#     ),
#     "m_{f_{0}(500)}": 0.4,
#     "m_{f_{0}(980)}": 0.88,
#     "m_{f_{0}(1370)}": 1.22,
#     "m_{f_{0}(1500)}": 1.45,
#     "m_{f_{0}(1710)}": 1.83,
#     R"\Gamma_{f_{0}(500)}": 0.3,
#     R"\Gamma_{f_{0}(980)}": 0.1,
#     R"\Gamma_{f_{0}(1710)}": 0.3,
# }